# Batch segmentation with text prompts

[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeos/segment-geospatial/blob/main/docs/examples/text_prompts_batch.ipynb)
[![image](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/opengeos/segment-geospatial&urlpath=lab/tree/segment-geospatial/docs/examples/text_prompts_batch.ipynb&branch=main)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/segment-geospatial/blob/main/docs/examples/text_prompts_batch.ipynb)

This notebook shows how to generate object masks from text prompts with the Segment Anything Model (SAM). 

Make sure you use GPU runtime for this notebook. For Google Colab, go to `Runtime` -> `Change runtime type` and select `GPU` as the hardware accelerator. 

## Install dependencies

Uncomment and run the following cell to install the required dependencies.

In [ ]:
# %pip install segment-geospatial groundingdino-py leafmap localtileserver

In [ ]:
# pip install -U segment-geospatial

In [1]:
import leafmap
from samgeo import tms_to_geotiff, split_raster
from samgeo.text_sam import LangSAM

## Create an interactive map

In [2]:
m = leafmap.Map(center=[18.6086, -90.7414], zoom=18, height="800px")
m.add_basemap("SATELLITE")
m

Map(center=[18.6086, -90.7414], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

## Download a sample image

Pan and zoom the map to select the area of interest. Use the draw tools to draw a polygon or rectangle on the map

In [3]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = [-90.741, 18.5992, -90.7314, 18.6045]]

In [4]:
# m.user_roi_bounds()

[-90.741, 18.5992, -90.7314, 18.6045]

In [ ]:
# image = "Image.tif"
# tms_to_geotiff(output=image, bbox=bbox, zoom=19, source="Satellite", overwrite=True)

You can also use your own image. Uncomment and run the following cell to use your own image.

In [5]:
image = 'escarcega_ne.tif'

Display the downloaded image on the map.

In [6]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(bottom=7506365.000020371, center=[18.60368624964752, -90.73759546069091], controls=(ZoomControl(options=['…

## Split the image into tiles

In [13]:
split_raster(image, out_dir="tiles", tile_size=(4000, 4000), overlap=0)

## Initialize LangSAM class

The initialization of the LangSAM class might take a few minutes. The initialization downloads the model weights and sets up the model for inference.

In [14]:
sam = LangSAM()

final text_encoder_type: bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Specify text prompts

In [15]:
text_prompt = "tree"

## Segment images

Part of the model prediction includes setting appropriate thresholds for object detection and text association with the detected objects. These threshold values range from 0 to 1 and are set while calling the predict method of the LangSAM class.

`box_threshold`: This value is used for object detection in the image. A higher value makes the model more selective, identifying only the most confident object instances, leading to fewer overall detections. A lower value, conversely, makes the model more tolerant, leading to increased detections, including potentially less confident ones.

`text_threshold`: This value is used to associate the detected objects with the provided text prompt. A higher value requires a stronger association between the object and the text prompt, leading to more precise but potentially fewer associations. A lower value allows for looser associations, which could increase the number of associations but also introduce less precise matches.

Remember to test different threshold values on your specific data. The optimal threshold can vary depending on the quality and nature of your images, as well as the specificity of your text prompts. Make sure to choose a balance that suits your requirements, whether that's precision or recall.

In [16]:
sam.predict_batch(
    images='tiles', 
    out_dir='masks', 
    text_prompt=text_prompt, 
    box_threshold=0.24, 
    text_threshold=0.24,
    mask_multiplier=255, 
    dtype='uint8',
    merge=True,
    verbose=True
    )

Processing image 01 of 16: tiles\tile_0_0.tif...
Processing image 02 of 16: tiles\tile_0_1.tif...
Processing image 03 of 16: tiles\tile_0_2.tif...
Processing image 04 of 16: tiles\tile_0_3.tif...
Processing image 05 of 16: tiles\tile_1_0.tif...
Processing image 06 of 16: tiles\tile_1_1.tif...
Processing image 07 of 16: tiles\tile_1_2.tif...
Processing image 08 of 16: tiles\tile_1_3.tif...
Processing image 09 of 16: tiles\tile_2_0.tif...
Processing image 10 of 16: tiles\tile_2_1.tif...
Processing image 11 of 16: tiles\tile_2_2.tif...
Processing image 12 of 16: tiles\tile_2_3.tif...
Processing image 13 of 16: tiles\tile_3_0.tif...
Processing image 14 of 16: tiles\tile_3_1.tif...
Processing image 15 of 16: tiles\tile_3_2.tif...
Processing image 16 of 16: tiles\tile_3_3.tif...
Saved the merged prediction to masks\merged.tif.


## Visualize the results

In [17]:
m.add_raster('masks/merged.tif', cmap='viridis', nodata=0, layer_name='Mask')
m.add_layer_manager()
m

Map(bottom=30025752.0, center=[18.596212435711372, -90.72807264515728], controls=(ZoomControl(options=['positi…

![](https://i.imgur.com/JUhNkm6.png)